In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from python_files.data import GetData
from sklearn.model_selection import learning_curve
from python_files.basic_preprocessing import BasicPreprocessing
from python_files.preprocessing import Advancedprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import collections
import re
import itertools
import string
from functools import reduce
from datetime import datetime
import pickle

import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.base import TransformerMixin, BaseEstimator
from scipy import stats
from scipy.stats import pearsonr
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import FunctionTransformer
from wordcloud import WordCloud
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

/home/sourav/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
data = GetData().get_data()['AllMoviesDetailsCleaned']
data.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number,actor1_name,actor1_gender,actor2_name,actor2_gender,actor3_name,actor3_gender,actor4_name,actor4_gender,actor5_name,actor5_gender,actor_number,director_name,director_gender,director_number,producer_name,producer_number,screeplay_name,editor_name,belongs_to_collection
0,2,0.0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,21/10/1988,0.0,69.0,suomi,Released,NaN,Ariel,7.1,40,2,1,2,Turo Pajala,0,Susanna Haavisto,0.0,Matti Pellonpää,2,Eetu Hilkamo,0,none,0,4,Aki Kaurismäki,0.0,1,none,0,Aki Kaurismäki,Raija Talvio,NaN
1,3,0.0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",0.474450,Villealfa Filmproduction Oy,Finland,16/10/1986,0.0,76.0,English,Released,NaN,Shadows in Paradise,7.0,32,1,1,3,Matti Pellonpää,2,Kati Outinen,1.0,Sakari Kuosmanen,2,Esko Nikkari,2,Kylli Köngäs,0,7,Aki Kaurismäki,0.0,1,Mika Kaurismäki,1,Aki Kaurismäki,Raija Talvio,NaN
2,5,4000000.0,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698000,Miramax Films,United States of America,25/12/1995,4300000.0,98.0,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1,Tim Roth,2,Antonio Banderas,2.0,Jennifer Beals,1,Madonna,1,Marisa Tomei,1,24,Allison Anders,1.0,4,Lawrence Bender,1,none,Margaret Goodspeed,NaN
3,6,0.0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",1.322870,Universal Pictures,Japan,15/10/1993,12136938.0,110.0,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69,3,2,1,Emilio Estevez,2,Cuba Gooding Jr.,2.0,Denis Leary,2,Jeremy Piven,2,Peter Greene,2,15,Stephen Hopkins,2.0,1,Gene Levy,1,Lewis Colick,Tim Wellburn,NaN
4,8,42000.0,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,01/01/2006,0.0,80.0,English,Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,1,1,5,none,0,none,0.0,none,0,none,0,none,0,0,Timo Novotny,0.0,1,Timo Novotny,2,Michael Glawogger,Timo Novotny,NaN


In [4]:
df = data.copy()

In [6]:
df_cleaned = BasicPreprocessing().apply(df)
df_cleaned.head()

,id,budget,genres,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number,actor1_name,actor1_gender,actor2_name,actor2_gender,actor3_name,actor3_gender,actor4_name,actor4_gender,actor5_name,actor5_gender,actor_number,director_name,director_gender,director_number,producer_name,producer_number,screeplay_name,editor_name,collection,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
2,5,4000000.0,Crime|Comedy,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698000,Miramax Films,United States of America,25/12/1995,4300000.0,98.0,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1,Tim Roth,2,Antonio Banderas,2.0,Jennifer Beals,1,Madonna,1,Marisa Tomei,1,24,Allison Anders,1.0,4,Lawrence Bender,1,none,Margaret Goodspeed,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,11,11000000.0,Adventure|Action|Science Fiction,en,Star Wars,Princess Leia is captured and held hostage by ...,10.492614,Lucasfilm,United States of America,25/05/1977,775398007.0,121.0,"A long time ago in a galaxy far, far away...",Star Wars,8.0,6168,2,1,1,Mark Hamill,2,Harrison Ford,2.0,Carrie Fisher,1,Peter Cushing,2,Alec Guinness,2,106,George Lucas,2.0,1,Gary Kurtz,2,none,Marcia Lucas,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
7,12,94000000.0,Animation|Family,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",9.915573,Pixar Animation Studios,United States of America,30/05/2003,940335536.0,100.0,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,5531,1,1,1,Albert Brooks,2,Ellen DeGeneres,1.0,Alexander Gould,2,Willem Dafoe,2,Brad Garrett,2,24,Andrew Stanton,2.0,1,Graham Walters,1,Andrew Stanton,David Ian Salter,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,13,55000000.0,Comedy|Drama|Romance,en,Forrest Gump,A man with a low IQ has accomplished great thi...,10.351236,Paramount Pictures,United States of America,06/07/1994,677945399.0,142.0,"The world will never be the same, once you've ...",Forrest Gump,8.2,7204,1,1,1,Tom Hanks,2,Robin Wright,1.0,Gary Sinise,2,Mykelti Williamson,2,Sally Field,1,67,Robert Zemeckis,2.0,1,Wendy Finerman,3,Eric Roth,Arthur Schmidt,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
9,14,15000000.0,Drama,en,American Beauty,"Lester Burnham, a depressed suburban father in...",8.191009,DreamWorks SKG,United States of America,15/09/1999,356296601.0,122.0,Look closer.,American Beauty,7.9,2994,2,1,1,Kevin Spacey,2,Annette Bening,1.0,Thora Birch,1,Wes Bentley,2,Mena Suvari,1,41,Sam Mendes,2.0,1,Bruce Cohen,2,Alan Ball,Christopher Greenbury,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
X = df_cleaned.drop(columns = ['revenue'])
y = df_cleaned['revenue']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [9]:
X_train_processed, list_top_30 = Advancedprocessing.process(X_train)
X_test_processed, _ = Advancedprocessing.process(X_test, train_set=False)

/home/sourav/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:2224: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset[header_name] = dataset[col].apply(lambda x: 1 if item in x else 0)
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

/home/sourav/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:2224: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset[header_name] = dataset[col].apply(lambda x: 1 if item in x else 0)
/home/sourav/code/Urus1201/data-movie-revenue-prediction/python_files/preprocessing.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

In [11]:
X_train_processed.head()

,popularity,runtime,vote_average,vote_count,production_companies_number,production_countries_number,actor_number,director_number,producer_number,collection,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,week_sin,week_cos,release_age,budget_year_ratio,Steven Spielberg_name,Woody Allen_name,Clint Eastwood_name,Brian De Palma_name,Tim Burton_name,Robert Zemeckis_name,Ridley Scott_name,Alfred Hitchcock_name,Martin Scorsese_name,Oliver Stone_name,Steven Soderbergh_name,Tony Scott_name,Joel Schumacher_name,Robert Rodriguez_name,Wes Craven_name,Ron Howard_name,Renny Harlin_name,Barry Levinson_name,Richard Donner_name,Spike Lee_name,Richard Linklater_name,Walter Hill_name,Paul W.S. Anderson_name,Francis Ford Coppola_name,Peter Hyams_name,Roland Emmerich_name,Gore Verbinski_name,Sam Raimi_name,Ang Lee_name,Rob Reiner_name,Joel Silver_name,Brian Grazer_name,Jerry Bruckheimer_name,Scott Rudin_name,John Davis_name,Tim Bevan_name,Neal H. Moritz_name,Arnon Milchan_name,Roger Birnbaum_name,Michael Bay_name,Steve Golin_name,Tom Rosenberg_name,Judd Apatow_name,Luc Besson_name,James G. Robinson_name,Charles Roven_name,Albert R. Broccoli_name,Danny DeVito_name,Kevin Feige_name,John Hughes_name,Ronnie Screwvala_name,Gale Anne Hurd_name,Lorenzo di Bonaventura_name,Sydney Pollack_name,Mark Johnson_name,Walt Disney_name,Paramount Pictures_name,Universal Pictures_name,Twentieth Century Fox Film Corporation_name,Columbia Pictures_name,New Line Cinema_name,Walt Disney Pictures_name,Warner Bros._name,United Artists_name,Miramax Films_name,Metro-Goldwyn-Mayer (MGM)_name,Columbia Pictures Corporation_name,Village Roadshow Pictures_name,TriStar Pictures_name,DreamWorks SKG_name,Fox Searchlight Pictures_name,Summit Entertainment_name,Touchstone Pictures_name,Regency Enterprises_name,Lions Gate Films_name,Lionsgate_name,Imagine Entertainment_name,The Weinstein Company_name,Orion Pictures_name,BBC Films_name,Hollywood Pictures_name,Dimension Films_name,Lakeshore Entertainment_name,Lucasfilm_name,Fox 2000 Pictures_name,StudioCanal_name,United States of America_name,United Kingdom_name,India_name,France_name,Canada_name,Germany_name,Russia_name,Australia_name,Japan_name,Italy_name,China_name,Spain_name,Ireland_name,Mexico_name,Hong Kong_name,Belgium_name,Netherlands_name,Denmark_name,South Korea_name,Czech Republic_name,New Zealand_name,Switzerland_name,Brazil_name,Finland_name,Singapore_name,Argentina_name,Thailand_name,Norway_name,Sweden_name,Hungary_name
10256,0.353470,84.0,5.6,30,1,1,23,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.970942,0.239316,22.0,12000.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
187414,3.712921,134.0,7.0,1145,3,2,39,1,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.970942,-0.239316,6.0,19841.27,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
910,4.111459,143.0,7.3,2101,7,2,22,1,9,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.354605,0.935016,15.0,49850.45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6240,0.689956,144.0,6.9,136,1,2,30,3,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0.568065,0.822984,52.0,12936.55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36423,0.290329,99.0,3.6,7,1,2,1

In [12]:
# Create validation sets
X_train_processed_part, X_val, y_train_part, y_val = train_test_split(X_train_processed, 
                                                              y_train, 
                                                              test_size = 0.2,
                                                              random_state = 42)

# Print split result
print(X_train_processed_part.shape)
print(X_val.shape)
print(y_train_part.shape)
print(y_val.shape)

(3400, 150)
(850, 150)
(3400,)
(850,)


In [13]:
# transform the y test and train to log scale
y_train_part_log = y_train_part.apply(lambda x: np.log(x + 1))
y_val_log = y_val.apply(lambda x: np.log(x + 1))
y_test_log = y_test.apply(lambda x: np.log(x + 1))

In [11]:
# RobustScaler for rest of the numerical features
rbst_scaler = RobustScaler()
features = ['runtime', 'release_age', 'production_companies_number', 'production_countries_number', 'popularity', 'vote_average', 'vote_count',
            'actor_number', 'director_number', 'producer_number', 'release_age', 'budget_year_ratio'] #'spoken_languages_number'

X_train_processed_part[features] = rbst_scaler.fit_transform(X_train_processed_part[features])

X_val[features] = rbst_scaler.transform(X_val[features])

X_test_processed[features] = rbst_scaler.transform(X_test_processed[features])

In [14]:
import statsmodels.api as sm

In [15]:
def print_model_summary(column_name):
    X_ = sm.add_constant(X_train_processed_part[column_name])
    y_ = y_train_part_log
    
    model = sm.OLS(y_, X_).fit()
    
    if(model.rsquared > 0.01):
        print(model.summary())

In [26]:
columns = list(X_train_processed_part.columns)

for column in columns:
    print(column)
    print( print_model_summary(column))
    print("\n")

runtime


MissingDataError: exog contains inf or nans